### Load libraries

In [47]:
#load libraries
import pandas as pd
import numpy as np
import matplotlib as plt
import os
import seaborn as sns
import geopandas as gpd

### Read data

In [48]:
#read the ICER data
icer = pd.read_excel("../data/raw/ICER_data.xlsx")

In [49]:
#read in the data for life expectancy
le_district = pd.read_csv("../data/processed/le_combined.csv")
le_nat = pd.read_csv("../data/processed/le_calc_national.csv")
df_map = gpd.read_file("../data/processed/high_res.shp")

In [50]:
#rename first column as treatment
icer = icer.rename(columns={"Unnamed: 0": "Treatment"})

In [51]:
icer.head()

,Treatment,burden,qaly_lost,healthcare_cost,payer_cost
0,non_vaccine,382416,184088.54,26618814,16066984
1,vaccine,264217,127189.68,352828922,345497041


### Calculate district-level ICER based on the available ICER (average) from 2020-2024 in Indonesia (from research paper), nationwide, adjusted by life expectancy to obtain district-level ICER, assuming intervention costs remain approximately the same (similar) --> life expectancy is to adjust based on QALYs. All prices are adjusted to 2018 USD based on PPP and discounted by 3%

In [52]:
le_district.head()

,District,Year,Life Expectancy
0,ACEH,2022,70.18
1,SIMEULUE,2022,65.48
2,ACEH SINGKIL,2022,67.65
3,ACEH SELATAN,2022,64.64
4,ACEH TENGGARA,2022,68.48


In [53]:
#rename as district_life_exp
le_district = le_district.rename(columns={"Life Expectancy": "district_le"})

In [54]:
le_nat.head()

,Year,Life Expectancy
0,2019,69.213148
1,2020,69.363809
2,2021,69.474243
3,2022,69.747513
4,2023,70.018191


In [55]:
#rename as nat_le
le_nat = le_nat.rename(columns={"Life Expectancy": "nat_le"})

In [56]:
#merge le_district and le_nat by year
le_final = pd.merge(le_district, le_nat, on = "Year")

#replace all the NA values

#make sure all the life expectancy values are all floats
le_final["district_le"] = le_final["district_le"].astype(float)
le_final["nat_le"] = le_final["nat_le"].astype(float)

#calculate the le_factor, which is a comparison of the district-level to the national-level life expectancy
le_final["le_factor"] = le_final["district_le"] / le_final["nat_le"]

In [57]:
#create another column in df_map to calculate the discounting factor for each district in dki jakarta and west java
df = pd.merge(df_map, le_final, on = ("District", "Year"))

In [58]:
#remove the district_le and nat_le columns
df_final = df.drop(columns = ["district_le", "nat_le"])

In [59]:
df_final["qaly_lost_non_vaccine"] = icer["qaly_lost"].iloc[0]
df_final["qaly_lost_vaccine"] = icer["qaly_lost"].iloc[1]
df_final["burden_non_vaccine"] = icer["burden"].iloc[0]
df_final["burden_vaccine"] = icer["burden"].iloc[1]
df_final["healthcare_cost_non_vaccine"] = icer["healthcare_cost"].iloc[0] / df_final["burden_non_vaccine"]
df_final["healthcare_cost_vaccine"] = icer["healthcare_cost"].iloc[1] / df_final["burden_vaccine"]
df_final["payer_cost_non_vaccine"] = icer["payer_cost"].iloc[0] / df_final["burden_non_vaccine"]
df_final["payer_cost_vaccine"] = icer["payer_cost"].iloc[1] / df_final["burden_vaccine"]


In [60]:
# Scale total cost to produce district-level cost per year based on cases per district per year / total num of cases over 2020-2024 nationwide, assuming that healthcare costs for dengue treatment is linearly related to the number of dengue cases per year
# this cost factor measures the scale of the district-level, yearly dengue cases to the national-level dengue cases in total from 2020-2024.
df_final['cost_factor'] = df_final['Cases'] / (df_final['burden_non_vaccine'] + df_final['burden_vaccine'])


In [61]:
#calculate district-level adjusted ICER for payers and healthcare
# Scale QALYs lost using le_factor and the cost-factor to disaggregate the qalys to district-level, yearly measurements from the national-level total qalys lost from 2020-2024
df_final['qaly_le_scaled_vaccine'] = df_final['qaly_lost_vaccine'] * df_final['le_factor']/ df_final["burden_vaccine"]
df_final['qaly_le_scaled_nonvaccine'] = df_final['qaly_lost_non_vaccine'] * df_final['le_factor']/ df_final["burden_non_vaccine"]

df_final['qaly_lost_scaled_vaccine'] = df_final['qaly_le_scaled_vaccine'] * df_final['cost_factor']
df_final['qaly_lost_scaled_nonvaccine'] = df_final['qaly_le_scaled_nonvaccine'] * df_final['cost_factor']

In [62]:
# Scale costs
df_final['healthcare_cost_vaccine_scaled'] = df_final['healthcare_cost_vaccine'] * df_final['cost_factor']
df_final['healthcare_cost_nonvaccine_scaled'] = df_final['healthcare_cost_non_vaccine'] * df_final['cost_factor']
df_final['payer_cost_vaccine_scaled'] = df_final['payer_cost_vaccine'] * df_final['cost_factor']
df_final['payer_cost_nonvaccine_scaled'] = df_final['payer_cost_non_vaccine'] * df_final['cost_factor']
df_final.head()

,District,Year,Area_sq_km,HDI,Province,Cases,Population,Pop_den,urbanicity,geometry,...,payer_cost_vaccine,cost_factor,qaly_le_scaled_vaccine,qaly_le_scaled_nonvaccine,qaly_lost_scaled_vaccine,qaly_lost_scaled_nonvaccine,healthcare_cost_vaccine_scaled,healthcare_cost_nonvaccine_scaled,payer_cost_vaccine_scaled,payer_cost_nonvaccine_scaled
0,BANDUNG,2020,1767.96,72.39,Jawa Barat,2295,3623790,2049.701351,0.123155,"POLYGON ((107.73309 -6.814, 107.73354 -6.81427...",...,1307.626084,0.003549,0.510297,0.510296,0.001811,0.001811,4.739453,0.247046,4.640966,0.149116
1,BANDUNG,2021,1767.96,72.73,Jawa Barat,2002,3665655,2073.381185,0.123155,"POLYGON ((107.73309 -6.814, 107.73354 -6.81427...",...,1307.626084,0.003096,0.510802,0.510802,0.001581,0.001581,4.134373,0.215506,4.048459,0.130078
2,BANDUNG,2022,1767.96,73.16,Jawa Barat,4191,3707523,2097.062716,0.123155,"POLYGON ((107.73309 -6.814, 107.73354 -6.81427...",...,1307.626084,0.006481,0.510802,0.510802,0.003311,0.003311,8.654923,0.451141,8.475072,0.272307
3,BANDUNG,2023,1767.96,73.74,Jawa Barat,1005,3749391,2120.744248,0.123155,"POLYGON ((107.73309 -6.814, 107.73354 -6.81427...",...,1307.626084,0.001554,0.510615,0.510615,0.000794,0.000794,2.075447,0.108183,2.032319,0.065299
4,BANDUNG BARAT,2020,1305.77,68.08,Jawa Barat,966,1788336,1369.564318,0.079001,"POLYGON ((107.40945 -6.68851, 107.40986 -6.688...",...,1307.626084,0.001494,0.502038,0.502038,0.000750,0.000750,1.994907,0.103985,1.953452,0.062765


In [63]:
# Compute incremental costs per person!
df_final['incremental_healthcare_cost'] = (df_final['healthcare_cost_vaccine_scaled']) - (df_final['healthcare_cost_nonvaccine_scaled'])
df_final['incremental_payer_cost'] = (df_final['payer_cost_vaccine_scaled']) - (df_final['payer_cost_nonvaccine_scaled'])

df_final.head()

,District,Year,Area_sq_km,HDI,Province,Cases,Population,Pop_den,urbanicity,geometry,...,qaly_le_scaled_vaccine,qaly_le_scaled_nonvaccine,qaly_lost_scaled_vaccine,qaly_lost_scaled_nonvaccine,healthcare_cost_vaccine_scaled,healthcare_cost_nonvaccine_scaled,payer_cost_vaccine_scaled,payer_cost_nonvaccine_scaled,incremental_healthcare_cost,incremental_payer_cost
0,BANDUNG,2020,1767.96,72.39,Jawa Barat,2295,3623790,2049.701351,0.123155,"POLYGON ((107.73309 -6.814, 107.73354 -6.81427...",...,0.510297,0.510296,0.001811,0.001811,4.739453,0.247046,4.640966,0.149116,4.492407,4.491851
1,BANDUNG,2021,1767.96,72.73,Jawa Barat,2002,3665655,2073.381185,0.123155,"POLYGON ((107.73309 -6.814, 107.73354 -6.81427...",...,0.510802,0.510802,0.001581,0.001581,4.134373,0.215506,4.048459,0.130078,3.918867,3.918381
2,BANDUNG,2022,1767.96,73.16,Jawa Barat,4191,3707523,2097.062716,0.123155,"POLYGON ((107.73309 -6.814, 107.73354 -6.81427...",...,0.510802,0.510802,0.003311,0.003311,8.654923,0.451141,8.475072,0.272307,8.203782,8.202765
3,BANDUNG,2023,1767.96,73.74,Jawa Barat,1005,3749391,2120.744248,0.123155,"POLYGON ((107.73309 -6.814, 107.73354 -6.81427...",...,0.510615,0.510615,0.000794,0.000794,2.075447,0.108183,2.032319,0.065299,1.967263,1.967020
4,BANDUNG BARAT,2020,1305.77,68.08,Jawa Barat,966,1788336,1369.564318,0.079001,"POLYGON ((107.40945 -6.68851, 107.40986 -6.688...",...,0.502038,0.502038,0.000750,0.000750,1.994907,0.103985,1.953452,0.062765,1.890922,1.890687


In [64]:
# Compute incremental QALY
df_final['incremental_qaly'] = (df_final['qaly_lost_scaled_vaccine']) - (df_final['qaly_lost_scaled_nonvaccine'])

In [65]:
#compute NMB per capita / case at the district level
df_final['nmb_healthcare_observed'] = 3 * 3859 * df_final["incremental_qaly"] - df_final["incremental_healthcare_cost"]
df_final['nmb_payer_observed'] = 3 * 3859 * df_final["incremental_qaly"] - df_final["incremental_payer_cost"]

In [66]:
#only retain the necessary rows and columns, save it as the benchmark ICER calculations
df_final = df_final[["District", "Province", "Cases", "Population", "Year", "nmb_healthcare_observed", "nmb_payer_observed", "burden_non_vaccine", "burden_vaccine", "qaly_le_scaled_vaccine", "qaly_le_scaled_nonvaccine", 'healthcare_cost_non_vaccine', 'healthcare_cost_vaccine', 'payer_cost_non_vaccine', 'payer_cost_vaccine']]

In [67]:
df_final.head()

,District,Province,Cases,Population,Year,nmb_healthcare_observed,nmb_payer_observed,burden_non_vaccine,burden_vaccine,qaly_le_scaled_vaccine,qaly_le_scaled_nonvaccine,healthcare_cost_non_vaccine,healthcare_cost_vaccine,payer_cost_non_vaccine,payer_cost_vaccine
0,BANDUNG,Jawa Barat,2295,3623790,2020,-4.492387,-4.491830,382416,264217,0.510297,0.510296,69.606957,1335.375551,42.014414,1307.626084
1,BANDUNG,Jawa Barat,2002,3665655,2021,-3.918849,-3.918364,382416,264217,0.510802,0.510802,69.606957,1335.375551,42.014414,1307.626084
2,BANDUNG,Jawa Barat,4191,3707523,2022,-8.203745,-8.202728,382416,264217,0.510802,0.510802,69.606957,1335.375551,42.014414,1307.626084
3,BANDUNG,Jawa Barat,1005,3749391,2023,-1.967255,-1.967011,382416,264217,0.510615,0.510615,69.606957,1335.375551,42.014414,1307.626084
4,BANDUNG BARAT,Jawa Barat,966,1788336,2020,-1.890913,-1.890679,382416,264217,0.502038,0.502038,69.606957,1335.375551,42.014414,1307.626084


In [68]:
df_final["nmb_healthcare_observed"] > 0

0      False
1      False
2      False
3      False
4      False
       ...  
127    False
128    False
129    False
130    False
131    False
Name: nmb_healthcare_observed, Length: 132, dtype: bool

### Save the df for other icer calculations

In [69]:
df_final.to_csv("../data/processed/nmb_observed.csv")